<a href="https://colab.research.google.com/github/jsgovea/PyTorch-learning/blob/main/cnn_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, sampler
import torchvision.datasets as datasets
import torchvision.transforms as T
import matplotlib.pyplot as plt


In [9]:
DATA_PATH = '/content/cifar-10'
NUM_TRAIN = 50000
NUM_VAL = 5000
NUM_TEST = 5000
MINBATCH_SIZE = 64

transform_cifar = T.Compose([
    T.ToTensor(),
    T.Normalize([0.491, 0.482, 0.447], [0.247, 0.243, 0.261])
])

In [10]:
# Train dataset

# Download the dataset
cifar10_train = datasets.CIFAR10(DATA_PATH, train=True, download=True,
                                 transform=transform_cifar)
train_loader = DataLoader(cifar10_train, batch_size=MINBATCH_SIZE,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))


100%|██████████| 170498071/170498071 [00:09<00:00, 17894263.84it/s]


Extracting /content/cifar-10/cifar-10-python.tar.gz to /content/cifar-10
